# TITLE: Preprocessing Scrapper 

## Information: 
This file converts commit ID and Project names found in excel file **Neutral_Breaker.xlsx** 
to urls and scraps the urls for path names with the key words -- **test**, **util**, **build**. 
It then creates a two pandas data frames neutral and breaker with the columns 
1. Project_Name 
2. Commit_ID 
3. Commit_URL 
4. Testing -- Binary 
5. Build -- Binary 
6. Maintenance -- BInary 

In [25]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError
from urllib.error import URLError
from os import path 
import os 
import xlrd
import re
import pandas as pd

In [13]:
def load_data(file_path, sheet_name):
    data = pd.read_excel(file_path, sheet_name=sheet_name)
    return data

In [14]:
def create_url(data):
    """

    :param data: panda data frame
    :return: url column created by concatenating 'Project_Name' and 'Commit_ID'
    """
    url_list = []
    for ii in range(len(data)):
        split_proj_name = data["Project_Name"].loc[ii].split("-", 1)
        organization = split_proj_name[0]
        project = split_proj_name[1]
        url_list.append("https://github.com/" + organization + "/" + project + "/commit/" +
                        str(data["Commit_ID"].loc[ii]))
    data["Commit_URL"] = url_list
    return data

In [26]:
file_path = ''
if not path.exists("Neutral_Breaker.xlsx"):
    print("upload 'Neutral_Breaker.xlsx' to current working directory!")
else:
    pwd = os.getcwd()
    file_path += pwd + '\\Neutral_Breaker.xlsx'
neutral = load_data(file_path, "Neutral_url")
breaker = load_data(file_path, "Breaker_url")
neutral = create_url(neutral)
breaker = create_url(breaker)
print(neutral.head(10))
print(" ")
print(breaker.head(10))

     Project_Name Commit_ID                                        Commit_URL
0     apache-avro   126e976     https://github.com/apache/avro/commit/126e976
1     apache-avro   2df0775     https://github.com/apache/avro/commit/2df0775
2     apache-avro   a39e6de     https://github.com/apache/avro/commit/a39e6de
3     apache-avro   2020c8a     https://github.com/apache/avro/commit/2020c8a
4     apache-avro   fa0059c     https://github.com/apache/avro/commit/fa0059c
5     apache-avro   50baf4c     https://github.com/apache/avro/commit/50baf4c
6  apache-calcite   44c72c3  https://github.com/apache/calcite/commit/44c72c3
7  apache-calcite   c2059f1  https://github.com/apache/calcite/commit/c2059f1
8  apache-calcite   2c339be  https://github.com/apache/calcite/commit/2c339be
9  apache-calcite   e117c10  https://github.com/apache/calcite/commit/e117c10
 
          Project_Name Commit_ID  \
0       apache-calcite   0b9ea98   
1       apache-calcite   980d9f8   
2       apache-calcite   e9d5060

In [16]:
def parse_urls(data):
    """
    parses the url within data["Commit_URL"] and classifies the commit
    as either 'Build', 'Testing', 'Maintenance' based on changes to files in diffs.

    :param data: pandas data frame
    :return: pandas data frame with three new binary columns 'Testing', 'Maintenance', 'Build'
    """
    testing = [0] * len(data["Commit_URL"])
    build = [0] * len(data["Commit_URL"])
    maintenance = [0] * len(data["Commit_URL"])
    for ii in range(len(data["Commit_URL"])):
        try:
            html = urlopen(data["Commit_URL"].iloc[ii])
            bsObj = BeautifulSoup(html, "html.parser")
            paths = bsObj.findAll("a", {"href": re.compile(r"#diff-[a-z0-9]+")})
            for path in paths:
                if len(path.attrs) == 1:
                    if re.match(r".*(build|pom).*", str(path)):
                        build[ii] = 1
                    if re.match(r".*(test|tests|tester).*", str(path)):
                        testing[ii] = 1
                    if re.match(r".*(u|U)til.*", str(path)) or re.match(r".*(h|H)elper.*", str(path)):
                        maintenance[ii] = 1
        except HTTPError as e:
            print(data["Commit_ID"].iloc[ii])
        except URLError as e:
            print("The server could not be found!")
    data["Testing"] = testing
    data["Build"] = build
    data["Maintenance"] = maintenance
    return data


In [17]:
neutral = parse_urls(neutral)
breaker = parse_urls(breaker)

In [21]:
print(neutral.head(11))

      Project_Name Commit_ID  \
0      apache-avro   126e976   
1      apache-avro   2df0775   
2      apache-avro   a39e6de   
3      apache-avro   2020c8a   
4      apache-avro   fa0059c   
5      apache-avro   50baf4c   
6   apache-calcite   44c72c3   
7   apache-calcite   c2059f1   
8   apache-calcite   2c339be   
9   apache-calcite   e117c10   
10  apache-calcite   3c32deb   

                                          Commit_URL  Testing  Build  \
0      https://github.com/apache/avro/commit/126e976        1      0   
1      https://github.com/apache/avro/commit/2df0775        1      0   
2      https://github.com/apache/avro/commit/a39e6de        1      0   
3      https://github.com/apache/avro/commit/2020c8a        1      0   
4      https://github.com/apache/avro/commit/fa0059c        0      0   
5      https://github.com/apache/avro/commit/50baf4c        1      0   
6   https://github.com/apache/calcite/commit/44c72c3        1      0   
7   https://github.com/apache/calcite/c

In [23]:
print(breaker.head(11))

           Project_Name Commit_ID  \
0        apache-calcite   0b9ea98   
1        apache-calcite   980d9f8   
2        apache-calcite   e9d5060   
3        apache-calcite   a63639b   
4        apache-calcite   a611d64   
5        apache-calcite   c711fed   
6        apache-calcite   5cee2a1   
7   apache-commons-bcel   8ddaf50   
8   apache-commons-bcel   9bebece   
9   apache-commons-bcel   2010565   
10  apache-commons-bcel   e5585e4   

                                           Commit_URL  Testing  Build  \
0    https://github.com/apache/calcite/commit/0b9ea98        1      0   
1    https://github.com/apache/calcite/commit/980d9f8        0      0   
2    https://github.com/apache/calcite/commit/e9d5060        1      0   
3    https://github.com/apache/calcite/commit/a63639b        1      1   
4    https://github.com/apache/calcite/commit/a611d64        0      0   
5    https://github.com/apache/calcite/commit/c711fed        0      0   
6    https://github.com/apache/calcite/commi